In [1]:
!pip install tensorflow===2.12.0
!pip install torch
!pip install keras===2.10.0
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install nltk
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.10.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

##Running the standard NER Pipeline

In [3]:
from transformers import pipeline

input_text="Sam went to California on the 23rd of August. \
There, he visited Google headquarters with John Smith and bought a cap for $23"

basic_ner = pipeline("ner")

basic_ner(input_text)


[{'entity': 'I-PER',
  'score': 0.99887806,
  'index': 1,
  'word': 'Sam',
  'start': 0,
  'end': 3},
 {'entity': 'I-LOC',
  'score': 0.99972683,
  'index': 4,
  'word': 'California',
  'start': 12,
  'end': 22},
 {'entity': 'I-ORG',
  'score': 0.9960085,
  'index': 15,
  'word': 'Google',
  'start': 64,
  'end': 70},
 {'entity': 'I-PER',
  'score': 0.99891376,
  'index': 18,
  'word': 'John',
  'start': 89,
  'end': 93},
 {'entity': 'I-PER',
  'score': 0.99921584,
  'index': 19,
  'word': 'Smith',
  'start': 94,
  'end': 99}]

##Understanding the model architecture

In [4]:
#Print model architecture
print(basic_ner.model)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

## Reviewing model configuration

In [5]:
print(basic_ner.model.config)


BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_s

##Using a Custom Model and tokenizer

In [6]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates",
                                          from_pt=True)

model = TFAutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates",
                                                          from_pt=True)

print(model.config.id2label)

{0: 'O', 1: 'I-LOC', 2: 'I-PER', 3: 'I-MISC', 4: 'I-ORG', 5: 'I-DATE'}


In [7]:
#Prediction
enhanced_ner = pipeline('ner',
                        model=model,
                        tokenizer=tokenizer,
                        aggregation_strategy="simple")
enhanced_ner(input_text)

[{'entity_group': 'PER',
  'score': 0.9776213,
  'word': 'Sam',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9936407,
  'word': 'California',
  'start': 11,
  'end': 22},
 {'entity_group': 'DATE',
  'score': 0.9235597,
  'word': 'August',
  'start': 37,
  'end': 44},
 {'entity_group': 'ORG',
  'score': 0.5721672,
  'word': 'Google',
  'start': 63,
  'end': 70},
 {'entity_group': 'PER',
  'score': 0.9938346,
  'word': 'John Smith',
  'start': 88,
  'end': 99},
 {'entity_group': 'DATE',
  'score': 0.6406405,
  'word': '23',
  'start': 122,
  'end': 124}]